In [1]:
import pandas as pd 
import os 

from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

import seaborn as sns

from datasets import load_dataset
import numpy as np
import json

In [2]:
pwd

'/home/al2644/research/codebase/reasoning/perturb-r/notebook'

In [4]:
cd /home/al2644/research/codebase/reasoning/perturb-r

/home/al2644/research/codebase/reasoning/perturb-r


In [12]:
from utils import process_thinking as tools

# Reasoning Results

In [216]:
root = "./results/math-500/benchmark"
for fname in os.listdir(root):
    if 'correct' not in fname:
        print(fname)
        df = pd.read_pickle(os.path.join(root, fname))
        print(f"Accuracy: {df['correct'].mean()}")

R1-Distill-Qwen-1.5B.pickle
Accuracy: 0.8413333333333334
R1-Distill-Qwen-7B.pickle
Accuracy: 0.9213333333333333
R1-Distill-Qwen-14B.pickle
Accuracy: 0.939
R1-Distill-Qwen-32B.pickle
Accuracy: 0.9473333333333334
AM-Distill-Qwen-32B.pickle
Accuracy: 0.9633333333333334
QwQ-32B.pickle
Accuracy: 0.9603333333333334
Qwen3-1.7B.pickle
Accuracy: 0.91
Qwen3-4B.pickle
Accuracy: 0.9483333333333334
Qwen3-8B.pickle
Accuracy: 0.9546666666666667
R1-Distill-Qwen3-8B.pickle
Accuracy: 0.897


In [220]:
df = pd.read_pickle(os.path.join(root, "R1-Distill-Qwen-1.5B.pickle"))
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

In [305]:
idx = 15
example = df[df["correct"] == 1.0].iloc[idx]
problem, response, solution = example["problem"], example["response"], example["solution"]
reasoning, answer = response.split("</think>")

In [17]:
input_str = tokenizer.apply_chat_template([{"role": "user", "content": problem}], tokenize=False, add_generation_prompt=True)

In [70]:
chunks = tools.minmax_chunk(reasoning, 30)
original_chunks = chunks[:5]
corrupted_chunks = chunks[5:10]
reasoning_prompt = "###ORIGINAL REASONING START\n"  + "\n\n".join(original_chunks) + "\n###ORIGINAL REASONING END\n"
reasoning_prompt += "###REASONING TO CHANGE START\n" + "\n\n".join(corrupted_chunks) + "\n###REASONING TO CHANGE END"

In [ ]:
stream = ""
for idx, chunk in enumerate(chunks):
    stream += ""

In [222]:
print(response)

Alright, so I've got this function here, f(x) = (3x - 2)/(x - 2), and I need to find the value of f(-2) + f(-1) + f(0). Hmm, okay. Let me see. I think I should just plug in each of those x-values into the function one by one and then add them up. Sounds straightforward, but let me make sure I don't make any mistakes.

Starting with f(-2). So, wherever there's an x, I'll substitute -2. That gives me (3*(-2) - 2) divided by (-2 - 2). Let me calculate the numerator first: 3 times -2 is -6, minus 2 is -8. Then the denominator is -2 - 2, which is -4. So, f(-2) is -8 divided by -4. Hmm, negative divided by negative is positive, so that's 2. Okay, that seems good.

Next up is f(-1). Plugging -1 into the function: (3*(-1) - 2) over (-1 - 2). Numerator: 3 times -1 is -3, minus 2 is -5. Denominator: -1 - 2 is -3. So, f(-1) is -5 divided by -3. Again, negatives cancel out, so that's 5/3. Got that.

Now, f(0). Plugging in 0: (3*0 - 2)/(0 - 2). Numerator: 0 minus 2 is -2. Denominator: 0 - 2 is -2. 

# Annotated df

In [286]:
root = "./results/math-500/annotated_reasoning/"
df = pd.read_pickle(os.path.join(root, "R1-Distill-Qwen-1.5B.pickle"))
os.listdir(root)

['R1-Distill-Qwen-1.5B.pickle', 'Qwen3-1.7B.pickle']

In [301]:
example = df.sample(1).iloc[0]
index = example["1st_answer_chunk_index"]
chunks = example["chunks"]
prompt = example["prompt"]
answer = example["answer"]
example

problem                      Let $f$ be defined by  \[f(x) = \left\{\n\begi...
solution                     The number $f^{-1}(0)$ is the value of $x$ suc...
answer                                                                       0
subject                                                                Algebra
level                                                                        5
unique_id                                               test/algebra/2277.json
response                     Okay, so I need to find the sum of the inverse...
correct                                                                    1.0
chunks                       [Okay, so I need to find the sum of the invers...
prompt                       <instruction>\nYou are given a sequence of rea...
idx                                                                       1980
1st_answer_annotation_raw                    [<answer_chunk>29</answer_chunk>]
1st_answer_chunk_index                              

In [304]:
print(prompt)

<instruction>
You are given a sequence of reasoning steps, divided into chunks, representing how a model solves a problem. Your task is to precisely locate the chunk number where the answer is first reached by the model. From the starting chunk, pinpoint what is the first chunk where model has reached the solution. You can ignore later chunks that verify, check, or try alternative methods, only mark the first chunk where the correct answer is mentioned by hinted. The answer usually should not appear in the first chunk that restates the problem. The answer should be fully derived but not necessarily verified yet.

You can start checking chunk by chunk from the start to the end. You should verify that it is indeed the first mention of the answer. At the end of your response, You MUST report the chunk number with the tags:
<answer_chunk>CHUNK_NUMBER</answer_chunk>
</instruction>

problem: Let $f$ be defined by  \[f(x) = \left\{
\begin{array}{cl}
3-x & \text{ if } x \leq 3, \\
-x^3+2x^2+3x

In [302]:
chunks[int(index)]

'\nTherefore, f inverse of 6 is -3.\n\nEarlier, I thought that x=3 is a solution to f(x)=0, but f(3)=0, not 6. So, f inverse of 0 is 3, and f inverse of 6 is -3. So, adding them together: 3 + (-3)=0.\n'

In [306]:
example

problem                      If $f(x) = \frac{3x-2}{x-2}$, what is the valu...
solution                     $f(-2)+f(-1)+f(0)=\frac{3(-2)-2}{-2-2}+\frac{3...
answer                                                            \frac{14}{3}
subject                                                                Algebra
level                                                                        3
unique_id                                               test/algebra/2584.json
response                     Alright, so I've got this function here, f(x) ...
correct                                                                    1.0
chunks                       [Alright, so I've got this function here, f(x)...
prompt                       <instruction>\nYou are given a sequence of rea...
idx                                                                         15
1st_answer_annotation_raw    [The correct answer \(\frac{14}{3}\) is first ...
1st_answer_chunk_index                              

In [ ]:
# 